路由

In [18]:
import io
import contextlib 
from ChatGLM3_router import ChatGLM
from langchain.chains.router import MultiPromptChain
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [19]:
# 主要创建三个知识域的模板，一个营销文案作者，一个客服，一个小红书

Copywriter_template = """You are a master of persuasion and a pro at captivating audiences. \
You have the ability to craft captivating messages that move people to action. \
You know that the key to effective communication is understanding your audience and speaking directly to their needs and desires.\

Here is a question:
{input}"""

RedBookWriter_template ="""You are a beauty blogger with a knack for discovering the latest trends and providing makeup tips. \
You have an eye for aesthetics and are always looking for ways to enhance your beauty routine. \
You are passionate about sharing your knowledge and inspiring others to embrace their beauty. 

Here is a question:
{input}"""

customerService_template = """You are a customer service representative. You are skilled at assisting customers and addressing their concerns. \
You take a problem-solving approach to help resolve issues and provide solutions. You should answer product, brand questions and purchase questions，and introduce product features.

Here is a question:
{input}"""

In [20]:
prompt_infos = [
    {
        "name": "Copywriter",
        "description": "Good for writing marketing copy ",
        "prompt_template": Copywriter_template,
    },
    {
        "name": "customerService",
        "description": "Good for answering product, brand, purchase and product features questions",
        "prompt_template": customerService_template,
    },
    {
        "name": "RedBookWriter",
        "description": "Good for writing Script title soft article advertising Weibo tweet ",
        "prompt_template": RedBookWriter_template,
    },
]

In [21]:
model_path = "/data/tds/ChatGLM3/chatglm3-6b"
llm = ChatGLM()
llm.load_model(model_name_or_path=model_path)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [22]:
# 下面这段主要通过遍历得到destination_chains
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
# 得到一个起始chain
default_chain = ConversationChain(llm=llm, output_key="text")

In [23]:
'''
#使用cohere
from langchain.chains.router.embedding_router import EmbeddingRouterChain
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Chroma
import os
os.environ["COHERE_API_KEY"]="JaYT27nDgJYh6yvsbknQdW4yYLuMTvNsvvBy7HVl"

52/27
names_and_descriptions = [
    ("Copywriter", ["营销文案写作任务"]),
    ("customerService", ["知识问答产品选择任务"]),
    ("RedBookWriter", ["直播视频脚本标题软文广告写作微博广告推文任务"]),
]

names_and_descriptions = [
    ("Copywriter", ["营销文案写作任务"]),
    ("customerService", ["知识问答产品选择任务"]),
    ("RedBookWriter", ["直播视频脚本标题广告写作微博广告推文任务美食美妆旅游推荐"]),
]
router_chain = EmbeddingRouterChain.from_names_and_descriptions(
    names_and_descriptions, Chroma, CohereEmbeddings(), routing_keys=["input"]
)
'''

'\n#使用cohere\nfrom langchain.chains.router.embedding_router import EmbeddingRouterChain\nfrom langchain.embeddings import CohereEmbeddings\nfrom langchain.vectorstores import Chroma\nimport os\nos.environ["COHERE_API_KEY"]="JaYT27nDgJYh6yvsbknQdW4yYLuMTvNsvvBy7HVl"\n\n52/27\nnames_and_descriptions = [\n    ("Copywriter", ["营销文案写作任务"]),\n    ("customerService", ["知识问答产品选择任务"]),\n    ("RedBookWriter", ["直播视频脚本标题软文广告写作微博广告推文任务"]),\n]\n\nnames_and_descriptions = [\n    ("Copywriter", ["营销文案写作任务"]),\n    ("customerService", ["知识问答产品选择任务"]),\n    ("RedBookWriter", ["直播视频脚本标题广告写作微博广告推文任务美食美妆旅游推荐"]),\n]\nrouter_chain = EmbeddingRouterChain.from_names_and_descriptions(\n    names_and_descriptions, Chroma, CohereEmbeddings(), routing_keys=["input"]\n)\n'

In [24]:
from langchain.chains.router.embedding_router import EmbeddingRouterChain
from langchain.vectorstores import Chroma
'''
47/31
names_and_descriptions = [
    ("Copywriter", ["营销文案写作任务"]),
    ("customerService", ["知识问答产品选择任务"]),
    ("RedBookWriter", ["直播视频脚本标题软文广告写作微博广告推文任务"]),
]
'''
'''
35/27
names_and_descriptions = [
    ("Copywriter", ["营销文案写作任务"]),
    ("customerService", ["知识问答产品选择任务"]),
    ("RedBookWriter", ["直播视频脚本标题广告写作微博广告推文任务美食美妆旅游推荐"]),
]
'''
names_and_descriptions = [
    ("Copywriter", ["营销文案写作任务"]),
    ("customerService", ["知识问答产品选择任务"]),
    ("RedBookWriter", ["直播视频脚本标题软文广告写作微博广告推文任务"]),
]
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")
embed_model = LangchainEmbedding(embeddings)
router_chain = EmbeddingRouterChain.from_names_and_descriptions(
    names_and_descriptions, Chroma, embed_model, routing_keys=["input"]
)



In [25]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [26]:
import pandas as pd
df = pd.read_excel("/home/kit/kit/ChatGLM3/langchain_demo/NewPromptRouter.xlsx",header=None)
df = df.assign(direct = None,extract = None)
df

,0,direct,extract
0,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,None,None
1,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,None,None
2,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,None,None
3,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,None,None
4,你是一名广告文案创作者，请根据以下要求创作宣传语：\n\n产品名称： LOHO新款眼睛\n\...,None,None
...,...,...,...
115,帮我写个文案，小红书风格的，汤臣倍健BYHEALTH品牌的益生菌，含有益微生物，可以保证肠道...,None,None
116,帮我写个文案，小红书风格的，乐力Osteoform品牌的益生菌，主要用于腹泻腹痛、受凉感冒、...,None,None
117,帮我写个文案，小红书风格的，迪辅乐Dipro品牌的益生菌，改善乳糖不耐受、改善肠道菌群结构、...,None,None
118,帮我写个文案，小红书风格的，圣歌兰Life's Flora品牌的益生菌，可以降低胆固醇、预防...,None,None


In [27]:
#输入user_question,判断句子路由方向，返回1代表生成文案，返回0代表知识问答
def get_direct(user_question,chain):
    with contextlib.redirect_stdout(io.StringIO()) as f:
        print(
            chain.run(
                user_question
            )
        )
        ans = f.getvalue() 
    sign = ans[51:61]
    if sign == "Copywriter":
        return 1
    else:
        return 0

In [28]:
for i in range(int(len(df))):
        direct = get_direct(df.iloc[i,0],chain)
        df.iloc[i,1] = direct
df

,0,direct,extract
0,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,None
1,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,None
2,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,None
3,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,None
4,你是一名广告文案创作者，请根据以下要求创作宣传语：\n\n产品名称： LOHO新款眼睛\n\...,1,None
...,...,...,...
115,帮我写个文案，小红书风格的，汤臣倍健BYHEALTH品牌的益生菌，含有益微生物，可以保证肠道...,0,None
116,帮我写个文案，小红书风格的，乐力Osteoform品牌的益生菌，主要用于腹泻腹痛、受凉感冒、...,1,None
117,帮我写个文案，小红书风格的，迪辅乐Dipro品牌的益生菌，改善乳糖不耐受、改善肠道菌群结构、...,1,None
118,帮我写个文案，小红书风格的，圣歌兰Life's Flora品牌的益生菌，可以降低胆固醇、预防...,1,None


In [29]:
df_ex = df[df['direct'] != 0].reset_index(drop=True)
df_ex

,0,direct,extract
0,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,None
1,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,None
2,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,None
3,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,None
4,你是一名广告文案创作者，请根据以下要求创作宣传语：\n\n产品名称： LOHO新款眼睛\n\...,1,None
5,你是一名广告文案创作者，请根据以下要求创作宣传语：\n\n产品名称： 潘多拉新款项链\n\n...,1,None
6,你是一名美食博主，请根据以下要求创作一篇小红书种草笔记： 文章主题：潮汕牛肉火锅探店的感受和...,1,None
7,你是一名美食博主，请根据以下要求创作一篇小红书种草笔记： 文章主题：韩式烤肉探店的感受和体验...,1,None
8,你是一个美妆博主，需要根据季节推荐适合的护肤品。\n\n季节： 夏季 \n\n皮肤类型： 干...,1,None
9,你是一个美妆博主，需要根据季节推荐适合的护肤品。\n\n季节： 秋季 \n\n皮肤类型： 干...,1,None


路由结果（32/61）错误路由29个
抽取

In [30]:
import torch
import time
import math
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig

In [31]:
def init_model():
    model = AutoModelForCausalLM.from_pretrained(
        "/data/tds/ChatGLM3/chatglm3-6b",
        torch_dtype=torch.float16,
        device_map="cuda:0",
        trust_remote_code=True
    )
    tokenizer = AutoTokenizer.from_pretrained(
        "/data/tds/ChatGLM3/chatglm3-6b",
        use_fast=False,
        trust_remote_code=True
    )
    return model, tokenizer


model, tokenizer = init_model()
model.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in_

In [32]:
prompt = """
请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、产品特点，以json格式返回，json格式为{"产品品类":,"产品品牌": ,"产品特点": }，注意：json格式不能改变，只用填入对应信息即可。
输出例子：{"产品品类": "内衣", "产品品牌": "帕兰朵", "产品特点": "小胸聚拢"}
{"产品品类": "美容仪", "产品品牌": "日立（HITACHI）", "产品特点": "提拉紧致,嫩肤淡纹,适合新手"}
{"产品品类": "益生菌", "产品品牌": "君乐宝", "产品特点": "能够改善消化功能、促进肠道蠕动、增加粪便量、改善便秘"}
可以返回空
帮我写个文案，小红书风格的，帕兰朵品牌的内衣，小胸聚拢特点的
"""
for i in range(len(df_ex)):
    user_question = df_ex.iloc[i,0]
    input = prompt.replace("帮我写个文案，小红书风格的，帕兰朵品牌的内衣，小胸聚拢特点的",user_question)
    response,history = model.chat(tokenizer, input, history=[])
    print(i,input,response)
    df_ex.iloc[i,2] = response

0 
请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、产品特点，以json格式返回，json格式为{"产品品类":,"产品品牌": ,"产品特点": }，注意：json格式不能改变，只用填入对应信息即可。
输出例子：{"产品品类": "内衣", "产品品牌": "帕兰朵", "产品特点": "小胸聚拢"}
{"产品品类": "美容仪", "产品品牌": "日立（HITACHI）", "产品特点": "提拉紧致,嫩肤淡纹,适合新手"}
{"产品品类": "益生菌", "产品品牌": "君乐宝", "产品特点": "能够改善消化功能、促进肠道蠕动、增加粪便量、改善便秘"}
可以返回空
你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：

产品名称： X品牌洗面奶 

目标用户： 20-35岁的都市女性 

内容要求： 自然融入产品参数和功能解读，结合用户故事或产品体验 

创作要点： 隐蔽推广，文章主题与产品关联不明显 
 {"产品品类": "护肤", "产品品牌": "X品牌", "产品特点": "温和洁面,深层清洁,保湿滋润"}
1 
请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、产品特点，以json格式返回，json格式为{"产品品类":,"产品品牌": ,"产品特点": }，注意：json格式不能改变，只用填入对应信息即可。
输出例子：{"产品品类": "内衣", "产品品牌": "帕兰朵", "产品特点": "小胸聚拢"}
{"产品品类": "美容仪", "产品品牌": "日立（HITACHI）", "产品特点": "提拉紧致,嫩肤淡纹,适合新手"}
{"产品品类": "益生菌", "产品品牌": "君乐宝", "产品特点": "能够改善消化功能、促进肠道蠕动、增加粪便量、改善便秘"}
可以返回空
你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：

产品名称： X品牌沐浴露 

目标用户： 20-35岁的都市女性 

内容要求： 自然融入产品参数和功能解读，结合用户故事或产品体验 

创作要点： 隐蔽推广，文章主题与产品关联不明显
 {"产品品类": "沐浴露", "产品品牌": "X品牌", "产品特点": "温和清洁，保湿滋润，消除疲劳，提神醒脑

使用Gemini来测试更多不同品类

In [47]:

import re  
def remove(text):  
    # 使用正则表达式删除阿拉伯数字、空格和英文字符  
    result = re.sub(r'[0-9\sA-Za-z.]', '', text)  
    return result

In [51]:
import google.generativeai as genai
genai.configure(api_key='')
genai_model = genai.GenerativeModel('gemini-pro')
prompt = """
改写以下句子，使其变为描述羽毛球的，使用真实的羽毛球品牌和特征，生成10条不一样的
君乐堡品牌的益生菌，能够改善消化功能、促进肠道蠕动、增肌粪便量、改善便秘
"""
genai_response = genai_model.generate_content(prompt)
print(str(genai_response.text).split('\n'))


['1. 提供稳定一致的飞行，即使在低风速条件下也能保持准确性，为球员提供更好的控制。', '2. 耐用性高，能够承受长时间的比赛，从而提供更长的使用寿命。', '3. 精准性高，能够以更高的精度和一致性落在指定区域，从而提高球员得分的机会。', '4. 采用独特的制造工艺，提供一致的重量和平衡，从而提高羽毛球的稳定性和准确性。', '5. 采用先进的材料和技术，为球员提供更佳的手感和控制力，从而提高比赛的整体水平。', '6. 提供更佳的空气动力学特性，能够更有效地切开空气，从而提高发球和扣杀的速度和力量。', '7. 为耐久性和可靠性而设计，能够承受长时间的比赛，从而为球员提供更长的使用寿命。', '8. 采用创新材料，为球员提供更佳的操控性，从而提高比赛的整体水平。', '9. 能够产生更响亮的击球声，从而提高击球的整体体验。', '10. 经过严格的质量控制，为球员提供更佳的品质保证，从而提高比赛的整体水平。']


In [52]:
prompt.replace('羽毛球','孕妇DHA')

'\n改写以下句子，使其变为描述孕妇DHA的，使用真实的孕妇DHA品牌和特征，生成10条不一样的\n君乐堡品牌的益生菌，能够改善消化功能、促进肠道蠕动、增肌粪便量、改善便秘\n'

In [57]:
bound_list = ['羽毛球','孕妇DHA','孕妇叶酸','孕妇多维复合维生素','乳母营养','洗衣液','投影仪','奶瓶','保温杯','精华液','水乳','面霜','气垫','口红','蛋白粉','面膜','眼霜','牙膏','蜜粉饼','白酒','面膜','遮瑕','巧克力','糖果']
prompt_list = []
for i in bound_list:
    prompt = """
    改写以下句子，使其变为描述羽毛球的，使用真实的羽毛球品牌和特征，生成10条不一样的
    君乐堡品牌的益生菌，能够改善消化功能、促进肠道蠕动、增肌粪便量、改善便秘
    """
    prompt = prompt.replace('羽毛球',i)
    genai_response = genai_model.generate_content(prompt)
    prompt_list = prompt_list + str(genai_response.text).split('\n')
prompt_list

['1. 尤尼克斯品牌的羽毛球，能够提升飞行稳定性、增强击球力度、提高球速、改善手感。',
 '2. 李宁品牌的羽毛球，能够降低风阻、增加球速、提高球的精准度、增强球的旋转。',
 '3. 亚狮龙品牌的羽毛球，能够减少球的振动、提高球的弹性、增强击球手感、改善球的旋转。',
 '4. 威克多品牌的羽毛球，能够延长击球时间、增强球的控制力、提高球的旋转、改善球的稳定性。',
 '5. 胜利品牌的羽毛球，能够提高球的耐久性、增强球的耐磨性、延长球的使用寿命、改善球的弹性。',
 '6. Kawasaki品牌的羽毛球，能够提高球的旋转、增强球的控制力、降低风阻、改善球的稳定性。',
 '7. RS LITE 3.0品牌的羽毛球，能够减震、防滑、耐磨、舒适、轻便，即使长久使用也不易损坏。',
 '8. RSL Tourney Feather品牌的羽毛球，采用天然羽毛制成，经过特殊加工处理，具有优异的飞行性能和耐用性，能够满足球员的各种需求。',
 '9. Forza Ti-1000品牌的羽毛球，采用纳米碳素纤维材料制成，具有优异的抗扭曲性能和击球手感，能够满足专业球员的各种需求。',
 '10. Yonex Aerosensa 300品牌的羽毛球，采用高弹性碳素纤维材料制成，具有优异的击球手感和稳定性，能够满足业余球员的各种需求。',
 '1. 惠氏孕妇DHA，富含对宝宝脑部发育至关重要的DHA和ARA，为宝宝提供完善的营养保障，助力宝宝的智力发育。',
 '',
 '',
 '2. 澳嘉宝孕妇DHA，含优质DHA，帮助宝宝们健康发展，促进智力发育，呵护宝宝的健康未来。',
 '',
 '',
 '3. 施恩菲孕妇DHA，含有丰富的DHA和EPA，有助于胎儿神经系统和视网膜的发育，支持胎儿健康成长，为未来学习打下坚实基础。',
 '',
 '',
 '4. 爱思诺孕妇DHA，含有纯正的DHA，可以促进胎儿脑部和视力发育，帮助宝宝健康成长，是孕期妈妈的理想选择。',
 '',
 '',
 "5. Nature's Way孕妇DHA，采用天然鱼油提取，含有丰富的DHA和EPA，有助于支持胎儿的大脑和眼睛发育，为宝宝提供全面的营养保障。",
 '',
 '',
 '6. Swisse孕妇DHA，富含优质DHA，有助于支持胎儿的大脑发育，为宝宝的智力发展提供营养支持，是孕期妈妈的明

In [61]:
prompt_list = [item for item in prompt_list if item != ""]  
import openpyxl  
# 创建一个新的工作簿  
workbook = openpyxl.Workbook()  
# 选择默认的工作表  
worksheet = workbook.active  
# 将列表中的值写入工作表的列中  
for i in range(len(prompt_list)):  
    worksheet.cell(row=i+1, column=1, value=prompt_list[i])  
# 保存工作簿  
workbook.save("MutilPrompt.xlsx")

In [71]:
df = pd.read_excel("/home/kit/kit/donson_intern/chatGLM/MutilPrompt.xlsx",header=None)
df = df.assign(direct = None,feature = None)
df

,0,direct,feature
0,尤尼克斯品牌的羽毛球，能够提升飞行稳定性、增强击球力度、提高球速、改善手感。,None,None
1,李宁品牌的羽毛球，能够降低风阻、增加球速、提高球的精准度、增强球的旋转。,None,None
2,亚狮龙品牌的羽毛球，能够减少球的振动、提高球的弹性、增强击球手感、改善球的旋转。,None,None
3,威克多品牌的羽毛球，能够延长击球时间、增强球的控制力、提高球的旋转、改善球的稳定性。,None,None
4,胜利品牌的羽毛球，能够提高球的耐久性、增强球的耐磨性、延长球的使用寿命、改善球的弹性。,None,None
...,...,...,...
235,真知棒棒糖酸甜可口，让人回味无穷，是童年最美好的回忆。,None,None
236,大白兔奶糖香甜浓郁，奶味十足，让你感受到儿时的味道。,None,None
237,徐福记水果糖口味酸甜，让你瞬间清爽，是夏天必备的零食。,None,None
238,旺旺糖果香脆可口，甜而不腻，让你享受酥脆的美味。,None,None


In [80]:
for i in range(int(len(df))):
    prompt1 = "帮我写个文案，小红书风格的，李宁品牌的羽毛球，能够增强羽毛球弹性、提高球速、增加旋转、改善手感。"
    prompt1 = prompt1.replace('李宁品牌的羽毛球，能够增强羽毛球弹性、提高球速、增加旋转、改善手感。',df.iloc[i,0])
    print(prompt1)
    direct = get_direct(prompt1,chain)
    print(direct)
    prompt2 = """
    请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、产品特点，以json格式返回，json格式为{"产品品类":,"产品品牌": ,"产品特点": }，注意：json格式不能改变，只用填入对应信息即可。
    输出例子：{"产品品类": "内衣", "产品品牌": "帕兰朵", "产品特点": "小胸聚拢"}
    可能的品类为：美容仪、益生菌、内衣、羽毛球、孕妇DHA、孕妇叶酸、孕妇多维复合维生素、乳母营养、洗衣液、投影仪、奶瓶、保温杯、精华液、水乳、面霜、气垫、口红、蛋白粉、面膜、眼霜、牙膏、蜜粉饼、白酒、面膜、遮瑕、巧克力、糖果
    帮我写个文案，小红书风格的，帕兰朵品牌的内衣，小胸聚拢特点的
    """
    input = prompt2.replace("帮我写个文案，小红书风格的，帕兰朵品牌的内衣，小胸聚拢特点的",prompt1)
    response,history = model.chat(tokenizer, input, history=[])
    print(response)
    df.iloc[i,1] = prompt1
    df.iloc[i,1] = direct
    df.iloc[i,2] = response

帮我写个文案，小红书风格的，尤尼克斯品牌的羽毛球，能够提升飞行稳定性、增强击球力度、提高球速、改善手感。
1
{
  "产品品类": "羽毛球",
  "产品品牌": "尤尼克斯",
  "产品特点": "能够提升飞行稳定性、增强击球力度、提高球速、改善手感"
}
帮我写个文案，小红书风格的，李宁品牌的羽毛球，能够降低风阻、增加球速、提高球的精准度、增强球的旋转。
1
{"产品品类": "羽毛球", "产品品牌": "李宁", "产品特点": "降低风阻、增加球速、提高球的精准度、增强球的旋转"}
帮我写个文案，小红书风格的，亚狮龙品牌的羽毛球，能够减少球的振动、提高球的弹性、增强击球手感、改善球的旋转。
1
{"产品品类": "羽毛球", "产品品牌": "亚狮龙", "产品特点": "减少球振动、提高球弹性、增强击球手感、改善球旋转"}
帮我写个文案，小红书风格的，威克多品牌的羽毛球，能够延长击球时间、增强球的控制力、提高球的旋转、改善球的稳定性。
1
{
    "产品品类": "羽毛球",
    "产品品牌": "威克多",
    "产品特点": "能够延长击球时间、增强球的控制力、提高球的旋转、改善球的稳定性"
}
帮我写个文案，小红书风格的，胜利品牌的羽毛球，能够提高球的耐久性、增强球的耐磨性、延长球的使用寿命、改善球的弹性。
1
{"产品品类": "体育用品", "产品品牌": "胜利", "产品特点": "羽毛球耐久性高,耐磨性增强,使用寿命延长,弹性改善"}
帮我写个文案，小红书风格的，Kawasaki品牌的羽毛球，能够提高球的旋转、增强球的控制力、降低风阻、改善球的稳定性。
1
{"产品品类": "羽毛球", "产品品牌": "Kawasaki", "产品特点": "提高球旋转、增强球控制力、降低风阻、改善球稳定性"}
帮我写个文案，小红书风格的，RS LITE 3.0品牌的羽毛球，能够减震、防滑、耐磨、舒适、轻便，即使长久使用也不易损坏。
1
{"产品品类": "体育用品", "产品品牌": "RS LITE 3.0", "产品特点": "减震、防滑、耐磨、舒适、轻便" }
帮我写个文案，小红书风格的，RSL Tourney Feather品牌的羽毛球，采用天然羽毛制成，经过特殊加工处理，具有优异的飞行性能和耐用性，能够满足球员的各种需求。

In [81]:
df.to_excel("./outputMutilPrompt.xlsx")

In [34]:
prompt1 = "帮我写个文案，小红书风格的，李宁品牌的羽毛球，能够增强羽毛球弹性、提高球速、增加旋转、改善手感。"
direct = get_direct(prompt1,chain)
print(direct)
prompt2 = """
请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、产品特点，以json格式返回，json格式为{"产品品类":,"产品品牌": ,"产品特点": }，注意：json格式不能改变，只用填入对应信息即可。
输出例子：{"产品品类": "内衣", "产品品牌": "帕兰朵", "产品特点": "小胸聚拢"}
{"产品品类": "美容仪", "产品品牌": "日立（HITACHI）", "产品特点": "提拉紧致,嫩肤淡纹,适合新手"}
{"产品品类": "益生菌", "产品品牌": "君乐宝", "产品特点": "能够改善消化功能、促进肠道蠕动、增加粪便量、改善便秘"}
可能存在的品类有：美容仪、益生菌、内衣、羽毛球、孕妇DHA、孕妇叶酸、孕妇多维复合维生素、乳母营养、洗衣液、投影仪、奶瓶、保温杯、精华液、水乳、面霜、气垫、口红、蛋白粉、面膜、眼霜、牙膏、蜜粉饼、白酒、面膜、遮瑕、巧克力、糖果
可以返回空
帮我写个文案，小红书风格的，帕兰朵品牌的内衣，小胸聚拢特点的
"""
input = prompt2.replace("帮我写个文案，小红书风格的，帕兰朵品牌的内衣，小胸聚拢特点的",prompt1)
response,history = model.chat(tokenizer, input, history=[])
print(response)

1
{"产品品类": "羽毛球", "产品品牌": "李宁", "产品特点": "能够增强羽毛球弹性、提高球速、增加旋转、改善手感"}


In [35]:
prompt2 = """
将下面句子改写为羽毛球品类的，使用真实的羽毛球品牌和其特点：
君乐宝品牌的益生菌，能够改善消化功能、促进肠道蠕动、增加粪便量、改善便秘
"""
response,history = model.chat(tokenizer, prompt2, history=[])
print(response)

羽博品牌的羽毛球，具有高品质、高性能和舒适性等特点，能够提升您的羽毛球技能，让您在比赛中脱颖而出。


In [36]:
df_ex.to_excel("./outputRouterNR.xlsx")